In [1]:
import nemo.collections.asr as nemo_asr

fine_tuned_model = "./models/parakeet-spring-lab-asr-task-wavs-finetuned/parakeet_finetuned.nemo"

asr_model = nemo_asr.models.ASRModel.restore_from(restore_path=fine_tuned_model)

[NeMo W 2025-07-18 08:07:33 nemo_logging:361] /app/src/pyenv/lib/python3.10/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
      warnings.warn("Can't initialize NVML")
    


[NeMo I 2025-07-18 08:07:39 mixins:181] Tokenizer SentencePieceTokenizer initialized with 1024 tokens


[NeMo W 2025-07-18 08:07:40 modelPT:181] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: ../data/springlab-asr-task-wavs/train_manifest.json
    sample_rate: 16000
    batch_size: 4
    shuffle: true
    trim_silence: true
    max_duration: 20.0
    min_duration: 0.1
    normalize_transcripts: true
    
[NeMo W 2025-07-18 08:07:40 modelPT:188] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: ../data/springlab-asr-task-wavs/validation_manifest.json
    sample_rate: 16000
    batch_size: 4
    shuffle: false
    normalize_transcripts: true
    


[NeMo I 2025-07-18 08:07:41 features:305] PADDING: 0
[NeMo I 2025-07-18 08:07:52 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}
[NeMo I 2025-07-18 08:07:52 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-07-18 08:07:52 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-07-18 08:07:52 rnnt_models:226] Using RNNT Loss : tdt
    Loss tdt_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0, 'durations': [0, 1, 2, 3, 4], 'sigma': 0.02, 'omega': 0.1}


[NeMo W 2025-07-18 08:07:52 label_looping_base:109] No conditional node support for Cuda.
    Cuda graphs with while loops are disabled, decoding speed will be slower
    Reason: CUDA is not available


[NeMo I 2025-07-18 08:07:53 save_restore_connector:282] Model EncDecRNNTBPEModel was successfully restored from /app/src/model/parakeet-spring-lab-asr-task-wavs-finetuned/parakeet_finetuned.nemo.


In [2]:
audio_file_path = "../data/2086-149220-0033.wav"

output = asr_model.transcribe([audio_file_path])
print(output[0].text)

Transcribing:   0%|                                                                                                                     | 0/1 [00:00<?, ?it/s][NeMo W 2025-07-18 08:07:53 nemo_logging:361] /app/src/pyenv/lib/python3.10/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
      warnings.warn(warn_msg)
    
Transcribing: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.15it/s]

well i dont wish to see it any more observed pebe turning away her eyes it is certainly very like the old portrait


In [3]:
import json

with open("../data/springlab-asr-task-wavs/test_manifest.json") as f:
    for line in f:
        data = json.loads(line)
        break

In [4]:
data

{'audio_filepath': '/app/data/springlab-asr-task-wavs/test_audio_0.wav',
 'duration': 6.41,
 'text': 'systems spectral technology enables the development of new mobile applications'}

### Multiple Files 

In [5]:
import os
import json
import nemo.collections.asr as nemo_asr
from tqdm import tqdm

def load_test_manifest(manifest_path):
    """Load test manifest and return list of samples"""
    samples = []
    
    print(f"Loading test manifest from: {manifest_path}")
    
    with open(manifest_path, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:  # Skip empty lines
                try:
                    sample = json.loads(line)
                    samples.append(sample)
                except json.JSONDecodeError as e:
                    print(f"Error parsing line: {line}")
                    print(f"Error: {e}")
    
    print(f"Loaded {len(samples)} samples from manifest")
    return samples

def inference_with_manifest(model_path, manifest_path):
    """Inference using test manifest with ground truth comparison"""
    
    print(f"Loading fine-tuned model from: {model_path}")
    asr_model = nemo_asr.models.ASRModel.restore_from(restore_path=model_path)
    print("Model loaded successfully!")
    

    test_samples = load_test_manifest(manifest_path)
    
    results = []
    
    # Process each sample from manifest
    print("Starting inference with ground truth comparison...")
    
    for sample in tqdm(test_samples, desc="Processing test samples"):
        # Extract information from manifest
        audio_filepath = sample.get('audio_filepath', '')
        original_text = sample.get('text', '')
        duration = sample.get('duration', 0)
        
        # Get filename for reference
        filename = os.path.basename(audio_filepath)

        output = asr_model.transcribe([audio_filepath])
        predicted_text = output[0].text
        
        
        # Store result with comparison
        result = {
            "filename": filename,
            "audio_filepath": audio_filepath,
            "original_text": original_text,
            "predicted_text": predicted_text,
            "duration": duration
        }
        results.append(result)
    
    return results

In [8]:
import nemo.collections.asr as nemo_asr

fine_tuned_model = "./models/parakeet-spring-lab-asr-task-wavs-finetuned/parakeet_finetuned.nemo"

test_manifest_path = "../data/springlab-asr-task-wavs/test_manifest.json"

total_results = inference_with_manifest(model_path = fine_tuned_model, manifest_path = test_manifest_path)

In [3]:
# import json

# output_dir = "../data/springlab-asr-task-wavs-test-data-v1.json"

# with open(output_dir, 'w', encoding='utf-8') as f:
#     json.dump(total_results, f, indent=2, ensure_ascii=False)

In [9]:
import json

output_dir = "../data/springlab-asr-task-wavs-test-data-v1.json"

with open(output_dir, 'r', encoding='utf-8') as f:
    loaded_data = json.load(f)

In [10]:
loaded_data[0]

{'filename': 'test_audio_0.wav',
 'audio_filepath': '/app/data/springlab-asr-task-wavs/test_audio_0.wav',
 'original_text': 'systems spectral technology enables the development of new mobile applications',
 'predicted_text': 'system spectral technology enables the development of new mobile applications',
 'duration': 6.41}

### Evaluation 

In [1]:
import json
import re
from typing import List, Dict, Tuple
import pandas as pd

In [3]:
def edit_distance(ref: List[str], hyp: List[str]) -> int:
    """Calculate edit distance between two sequences"""
    d = [[0] * (len(hyp) + 1) for _ in range(len(ref) + 1)]
    
    for i in range(len(ref) + 1):
        d[i][0] = i
    for j in range(len(hyp) + 1):
        d[0][j] = j
    
    for i in range(1, len(ref) + 1):
        for j in range(1, len(hyp) + 1):
            if ref[i-1] == hyp[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = min(d[i-1][j], d[i][j-1], d[i-1][j-1]) + 1
    
    return d[len(ref)][len(hyp)]

def word_error_rate(reference: str, hypothesis: str) -> float:
    """Calculate Word Error Rate"""
    ref_words = reference.strip().split()
    hyp_words = hypothesis.strip().split()
    
    if len(ref_words) == 0:
        return 0.0 if len(hyp_words) == 0 else 1.0
    
    errors = edit_distance(ref_words, hyp_words)
    return errors / len(ref_words)

def character_error_rate(reference: str, hypothesis: str) -> float:
    """Calculate Character Error Rate (excluding spaces)"""
    ref_chars = list(reference.replace(' ', ''))
    hyp_chars = list(hypothesis.replace(' ', ''))
    
    if len(ref_chars) == 0:
        return 0.0 if len(hyp_chars) == 0 else 1.0
    
    errors = edit_distance(ref_chars, hyp_chars)
    return errors / len(ref_chars)

def normalize_text(text: str) -> str:
    """Normalize text for comparison"""
    if not text:
        return ""
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text)
    
    # Remove leading/trailing whitespace
    text = text.strip()
    
    return text

def calculate_and_save_metrics(loaded_data: List[Dict], output_file: str):
    """
    Calculate WER and CER for each sample and save to CSV
    
    Args:
        loaded_data: List of dictionaries with 'original_text' and 'predicted_text' keys
        output_file: Output CSV filename
    """
    
    print(f"Processing {len(loaded_data)} samples...")
    
    results = []
    
    for i, item in enumerate(loaded_data):
        # Get original texts (non-normalized for display)
        original_text = item['original_text']
        predicted_text = item['predicted_text']
        
        # Normalize for calculation
        original_normalized = normalize_text(original_text)
        predicted_normalized = normalize_text(predicted_text)
        
        # Calculate metrics
        wer = word_error_rate(original_normalized, predicted_normalized)
        cer = character_error_rate(original_normalized, predicted_normalized)
        
        # Store result
        results.append({
            'original_text': original_text,
            'predicted_text': predicted_text,
            'CER': round(cer, 4),
            'WER': round(wer, 4)
        })
        
        # Print progress
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{len(loaded_data)} samples")
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False, encoding='utf-8')
    
    return df

def calculate_and_save_with_filename(loaded_data: List[Dict], output_file: str):
    """
    Calculate WER and CER for each sample and save to CSV (includes filename if available)
    
    Args:
        loaded_data: List of dictionaries with 'original_text' and 'predicted_text' keys
        output_file: Output CSV filename
    """
    
    print(f"Processing {len(loaded_data)} samples...")
    
    results = []
    
    for i, item in enumerate(loaded_data):
        # Get original texts (non-normalized for display)
        original_text = item['original_text']
        predicted_text = item['predicted_text']
        filename = item.get('filename', f'sample_{i}')
        
        # Normalize for calculation
        original_normalized = normalize_text(original_text)
        predicted_normalized = normalize_text(predicted_text)
        
        # Calculate metrics
        wer = word_error_rate(original_normalized, predicted_normalized)
        cer = character_error_rate(original_normalized, predicted_normalized)
        
        # Store result
        results.append({
            'filename': filename,
            'original_text': original_text,
            'predicted_text': predicted_text,
            'CER': round(cer, 4),
            'WER': round(wer, 4)
        })
        
        # Print progress
        if (i + 1) % 100 == 0:
            print(f"Processed {i + 1}/{len(loaded_data)} samples")
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(results)
    df.to_csv(output_file, index=False, encoding='utf-8')
    
    # Print summary statistics
    avg_wer = df['WER'].mean()
    avg_cer = df['CER'].mean()
    
    print(f"\nResults saved to: {output_file}")
    print(f"Summary Statistics:")
    print(f"  Average WER: {avg_wer:.4f} ({avg_wer*100:.2f}%)")
    print(f"  Average CER: {avg_cer:.4f} ({avg_cer*100:.2f}%)")
    print(f"  Total samples: {len(results)}")
    
    
    return df

In [5]:
output_dir1 = "../data/springlab-asr-task-wavs-test-data-v1.json"

with open(output_dir1, 'r', encoding='utf-8') as f:
    loaded_data1 = json.load(f)

In [6]:
# Example usage
if __name__ == "__main__":
    # data
    example_data = loaded_data1
    
    filename = "../data/springlab-asr-task-wavs-test-data-v1-wer_cer_results.csv"
    # Save results to CSV
    df = calculate_and_save_with_filename(example_data, output_file = filename)
    
    print("\nCSV file created successfully!")

Processing 800 samples...
Processed 100/800 samples
Processed 200/800 samples
Processed 300/800 samples
Processed 400/800 samples
Processed 500/800 samples
Processed 600/800 samples
Processed 700/800 samples
Processed 800/800 samples

Results saved to: ../data/springlab-asr-task-wavs-test-data-v1-wer_cer_results.csv
Summary Statistics:
  Average WER: 0.1091 (10.91%)
  Average CER: 0.0500 (5.00%)
  Total samples: 800

CSV file created successfully!


In [12]:
import pandas as pd

df = pd.read_csv('../data/springlab-asr-task-wavs-test-data-v1-wer_cer_results.csv')

avg_wer = df['WER'].mean()
avg_cer = df['CER'].mean()

print(f"Average WER: {avg_wer:.4f} ({avg_wer*100:.2f}%)")
print(f"Average CER: {avg_cer:.4f} ({avg_cer*100:.2f}%)")
print(f"Total samples: {df.shape[0]}")

Average WER: 0.1091 (10.91%)
Average CER: 0.0500 (5.00%)
Total samples: 800
